<a href="https://colab.research.google.com/github/fghfdjtyjgfd/ML-DL/blob/main/RAG_condo_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community langchain_openai

# Web scrapping

In [ ]:
import os
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import pprint

In [ ]:
if os.path.exists("condo_detail"):
  files = os.listdir("condo_detail")
  for file in files:
    os.remove(f"condo_detail/{file}")
  os.rmdir("condo_detail")

In [ ]:
if not os.path.exists("condo_detail"):
  os.mkdir("condo_detail")
else:
  print("Directory already exist")

condo_detail_path = os.path.join("condo_detail")

In [ ]:
def clean_text(txt):
  txt = txt.text.replace("\n", " ")
  txt = re.sub(r"[^a-zA-Z0-9\s,.]", "", txt)
  return txt

In [ ]:
# หมอชิต บางเขน ลาดพร้าว
page_urls =[
    "https://www.livinginsider.com/searchword_en/Condo/Rent/1/%E0%B8%AB%E0%B8%A1%E0%B8%AD%E0%B8%8A%E0%B8%B4%E0%B8%95.html",
    "https://www.livinginsider.com/searchword_en/all/Rent/1/%E0%B8%9A%E0%B8%B2%E0%B8%87%E0%B9%80%E0%B8%82%E0%B8%99.html",
    "https://www.livinginsider.com/searchword_en/all/Rent/1/%E0%B8%A5%E0%B8%B2%E0%B8%94%E0%B8%9E%E0%B8%A3%E0%B9%89%E0%B8%B2%E0%B8%A7.html"
    ]

i = 0
j = 1

for page_url in page_urls:
  i = 0
  page_content = requests.get(page_url)
  soup = BeautifulSoup(page_content.content, 'html.parser')
  posts = soup.find_all('div', class_="col-md-3 col-sm-4 col-xs-6")

  for post in posts:
    if i >= 10:
      continue
    link = post.find('a', class_="image-ratio-4-3").get('href')
    in_post_content = requests.get(link)
    in_post_soup = BeautifulSoup(in_post_content.content, 'html.parser')
    post_content = in_post_soup.find('p', class_="wordwrap")
    post_content = clean_text(post_content)

    condo_name = in_post_soup.find('span', class_="text_project_detail_green").text
    price = in_post_soup.find('span', class_="t-24 price-detail mt-0").text.replace('\n', '')

    condo_detail = f"{condo_name} {price} {post_content} {link}"
    with open(condo_detail_path + f"/condo_detail{j}.txt", "a") as f:
      f.write(condo_detail)
    i += 1
    j += 1

# Load and Split documents

In [ ]:
!pip install chromadb==0.4.14

/bin/bash: line 1: chromadb.__version__: command not found


In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, TextLoader

In [ ]:
loader = DirectoryLoader("condo_detail",
                         glob="*.txt",
                         show_progress=True,
                         loader_cls=TextLoader)
docs = loader.load()

100%|██████████| 30/30 [00:00<00:00, 5863.97it/s]


In [ ]:
len(docs)

30

In [ ]:
docs[5].page_content

'Ramkhamhaeng Nida, Seri Thai, Sukhapiban 2 ฿38,000/mo FOR RENT Home office for rent, Lat Phrao  Seri Thai   Located in Baan Klang Muang project, can register a company    SOLD Rent 38,000 baht  month   Rent reduced from 45,000 baht  month, super cool    SOLD OUT Contact number  AZ  AZ       Real Estate Office, Back Office, Home Studio, Home Office, Production House, YouTuber, Content Creator, TikTok Agent, Marketing   3storey townhouse, corner unit, 2car parking  Land area 52 square wah, area 152 square meters  Function 3 bedrooms, 3 bathrooms, 1 living room Garden behind the house, pets allowed  Nearby places  Nomklao Demonstration School  KhwanRiam Floating Market, Isan Kitchen, Soi Mistine  Lotus Sukhapiban 3, Makro Ramintra  Kasemrad Hospital Ramkhamhaeng  The Paseo Town Ramkhamhaeng  Saphan Sung Subdistrict, Siam Amazing Park  Golf Channel Center, Wat Bamphen Nuea  Wat Rat Satthatham Wat Bangchan        Contact number            Contact number  . .      .,  https://www.livinginsi

In [ ]:
if not os.path.exists("chroma"):
  os.mkdir("chroma")
else:
  print("Directory already exist")

persistent_dir = os.path.join("chroma")

Directory already exist


In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

vectordb = Chroma.from_documents(docs,
                                 embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"),
                                 persist_directory="chroma")

<ipython-input-14-2a1344cb1428>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"),
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

# Create RAG chain

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_history_aware_retriever

from langchain.prompts import MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
os.environ['OPENAI_API_KEY'] = ""

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 3, "fetch_k": 10})

In [ ]:
contextualize_system_prompt = """
    Given a chat history and the latest user question
    which might reference context in the chat history,
    formulate a standalone question which can be understood
    without the chat history. Do NOT answer the question,
    just reformulate it if needed and otherwise return as it is.
"""

retriever_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

histroy_aware_retriever = create_history_aware_retriever(llm=llm,
                                                         retriever=retriever,
                                                         prompt=retriever_prompt)

In [ ]:
system_prompt = """
You are an assistant for answering question about condominiums, apartments for rent.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, say that you don't know.
Use three sentences maximum, provide a url with answer and keep the answer concise.

{context}
"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

qa_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(histroy_aware_retriever, qa_chain)

In [ ]:
store = {}

def get_session(session_id: str) -> BaseChatMessageHistory:
  if session_id not in store:
    store[session_id] = ChatMessageHistory()
  return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(rag_chain,
                                                      get_session,
                                                      input_messages_key="input",
                                                      history_messages_key="chat_history",
                                                      output_messages_key="answer")

In [35]:
question = "Recommend condo near bts ratchayothin"

In [36]:
session_id = 1

answer = conversational_rag_chain.invoke(
              {"input": question},
              config={
                  "configurable": {"session_id": session_id}
              }
          )

In [37]:
answer["answer"]

'I recommend the ASPIRE Ratchayothin condo for rent at ฿13,000/month. It is a fully furnished 1 bed, 1 bath unit located near BTS Ratchayothin. For more information, you can visit https://www.livinginsider.com/livingdetail_en/2471485/FOR-RENT-Aspire-Ratchayothin-New-room-ready-to-move-in-convenient-transportation.html.'

In [40]:
retriever.invoke(question)

[Document(metadata={'source': 'condo_detail/condo_detail1.txt'}, page_content='BTS Residence ฿16,900/mo Condo for rent BTS Residence 1 bedroom free parking\n Near BTS Mo ChitMRT Chatuchak Park\n   \n 2 bedrooms, 1 living room, 1 bathroom\n \n Size 50 sq m.\n \n Price 16,900 baht\n \n Complete furniture and electrical appliances Ready to move in\n  Builtin kitchen set with mezzanine\n  2 new air conditioners, living room and bedroom\n  Electric stove with hood and microwave\n  2 door refrigerator\n  Smart TV\n  Front loading washing machine\n  Beds with mattresses, size 5 feet, 2 sets 2 bedrooms\n  2 sets of wardrobes 2 bedrooms\n  Dressing table\n  3 seat sofa\n  Dining table set with 4 chairs\n  TV shelf\n  \n   Room details\n  The project has a total of 25 floors the room is on the 24th floor\n  Room type City view, Chatuchak Park side\n   \n Floating furniture and electrical appliances Ready to move in Appears as in the picture\n   \n Project facilities\n  Parking\n  Swimming pool, 